In [32]:
import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser
import datetime 

In [2]:
from selenium.webdriver.chrome.service import Service

In [3]:
my_service = Service(executable_path='./chromedriver.exe')
browser = Browser('chrome', service=my_service)

In [4]:
# visit the url 
url = 'https://www.mshp.dps.missouri.gov/HP68/SearchAction'
#call the browers and go to the url
browser.visit(url)

In [38]:
# Effort - extract list of dates
date_list = [(d.text, datetime.datetime.strptime(d.text, "%B %d, %Y")) for d in browser.find_by_xpath("//*[@id='date']/option") if d.text !='']

In [42]:
dates_desc = sorted(date_list, key=lambda x: x[1], reverse=False)

In [46]:
d = dates_desc[1]

In [48]:
browser.find_by_value('Search').click()

In [55]:
table_list = pd.read_html(browser.html)

In [56]:
assert len(table_list) == 1

In [92]:
df_crsh_rpt_list = table_list[0]

In [93]:
df_crsh_rpt_list.head()
# Objective - need to create a key to tie Crash Report List with Report

,Report,Name,Age,Person City/State,Personal Injury,Safety Device,Date,Time,Crash County,Crash Location,Troop
0,View,"MILLER, KATHLEEN S",68,"HARRISON, AR",SERIOUS,YES,03/07/2022,9:15PM,CRAWFORD,WESTBOUND I-44 AT THE 201 MILE MARKER,I
1,View,"WIENING, JAMES D",50,"BLYTHEVILLE, AR",FATAL,YES,03/07/2022,9:15PM,CRAWFORD,WESTBOUND I-44 AT THE 201 MILE MARKER,I
2,View,"STALLINGS, MARK D",63,"FOUR SEASONS, MO",MODERATE,YES,03/07/2022,4:59PM,COLE,US 54 WESTBOUND APPROXIMATELY 36 FEET WEST OF ...,F
3,View,"GAVIGAN, DILLAN T",18,"SHERIDAN, MO",SERIOUS,NO,03/07/2022,4:41PM,WORTH,WALNUT ST EAST OF MO 246 IN SHERIDAN,H
4,View,"JUVENILE,",13,"SHERIDAN, MO",MODERATE,NO,03/07/2022,4:41PM,WORTH,WALNUT ST EAST OF MO 246 IN SHERIDAN,H


In [101]:
anchors = browser.find_by_tag("table").links.find_by_text('View')

In [94]:
assert len(anchors) == df_crsh_rpt_list.shape[0]

In [97]:
import uuid
append_key_data = []
for a in anchors:
    append_key_data.append((a['href'], a['href'].split('=')[-1], str(uuid.uuid4())))
#raw_link_name = anchors[0]['href']
#acc_rpt_num = raw_link_name.split('=')[-1]
df_crsh_rpt_keys = pd.DataFrame(data=append_key_data, columns=['raw_href', 'acc_rpt_num', 'acc_uuid'])
df_crsh_rpt_keys.head()

,raw_href,acc_rpt_num,acc_uuid
0,https://www.mshp.dps.missouri.gov/HP68/Acciden...,220116287,adc97c5b-3990-4b15-b78b-4c7706da994f
1,https://www.mshp.dps.missouri.gov/HP68/Acciden...,220116287,01c624ef-1198-4ac4-9149-86b648a24784
2,https://www.mshp.dps.missouri.gov/HP68/Acciden...,220115939,9887bb70-40e6-4845-bc4f-47ad09165c77
3,https://www.mshp.dps.missouri.gov/HP68/Acciden...,220115911,29d71a0c-b9b5-4ec8-8f04-aab27c73d699
4,https://www.mshp.dps.missouri.gov/HP68/Acciden...,220115911,f8e85eeb-ebca-4079-92bc-be34284ee287


In [98]:
df_crsh_rpt_full = pd.merge(df_crsh_rpt_keys, df_crsh_rpt_list, left_index=True, right_index=True)

In [99]:
df_crsh_rpt_full.head()

,raw_href,acc_rpt_num,acc_uuid,Report,Name,Age,Person City/State,Personal Injury,Safety Device,Date,Time,Crash County,Crash Location,Troop
0,https://www.mshp.dps.missouri.gov/HP68/Acciden...,220116287,adc97c5b-3990-4b15-b78b-4c7706da994f,View,"MILLER, KATHLEEN S",68,"HARRISON, AR",SERIOUS,YES,03/07/2022,9:15PM,CRAWFORD,WESTBOUND I-44 AT THE 201 MILE MARKER,I
1,https://www.mshp.dps.missouri.gov/HP68/Acciden...,220116287,01c624ef-1198-4ac4-9149-86b648a24784,View,"WIENING, JAMES D",50,"BLYTHEVILLE, AR",FATAL,YES,03/07/2022,9:15PM,CRAWFORD,WESTBOUND I-44 AT THE 201 MILE MARKER,I
2,https://www.mshp.dps.missouri.gov/HP68/Acciden...,220115939,9887bb70-40e6-4845-bc4f-47ad09165c77,View,"STALLINGS, MARK D",63,"FOUR SEASONS, MO",MODERATE,YES,03/07/2022,4:59PM,COLE,US 54 WESTBOUND APPROXIMATELY 36 FEET WEST OF ...,F
3,https://www.mshp.dps.missouri.gov/HP68/Acciden...,220115911,29d71a0c-b9b5-4ec8-8f04-aab27c73d699,View,"GAVIGAN, DILLAN T",18,"SHERIDAN, MO",SERIOUS,NO,03/07/2022,4:41PM,WORTH,WALNUT ST EAST OF MO 246 IN SHERIDAN,H
4,https://www.mshp.dps.missouri.gov/HP68/Acciden...,220115911,f8e85eeb-ebca-4079-92bc-be34284ee287,View,"JUVENILE,",13,"SHERIDAN, MO",MODERATE,NO,03/07/2022,4:41PM,WORTH,WALNUT ST EAST OF MO 246 IN SHERIDAN,H


In [102]:
# run report
anchors[0].click()

In [126]:
type(anchors[0])

splinter.driver.webdriver.WebDriverElement

In [103]:
# Variable number of tables - need to extract tables and table names
tables = browser.find_by_tag("table")

In [105]:
tables[0].find_by_tag("caption").text

'Crash Information'

In [112]:
table_names = [str.lower(t.find_by_tag("caption").text).replace(' ', '_').replace('.', '_') for t in tables]
print(table_names)

['crash_information', 'vehicle_information', 'injury_information', 'misc__information']


In [109]:
df_table_list = pd.read_html(browser.html)

In [120]:
crsh_rpt_details = []
for i, df in enumerate((df_table_list)):
    print(table_names[i])
#    print(df)
    df['table_name'] = table_names[i]
    crsh_rpt_details.append({f'{table_names[i]}':df})

crash_information
vehicle_information
injury_information
misc__information


In [124]:
crsh_rpt_details[1]

{'vehicle_information':    Veh. # Vehicle Description Damage                   Disposition  \
 0       1   2019 BUICK ENCORE  TOTAL  TOWED BY/TO SPURGEONS TOWING   
 
         Driver Name Driver Gender  Driver Age Safety Device Driver City/State  \
 0  WIENING, JAMES D          MALE          50           YES   BLYTHEVILLE, AR   
 
   Driver Insurance Vehicle Direction           table_name  
 0       STATE FARM         WESTBOUND  vehicle_information  }

In [123]:
browser.back()

In [125]:
browser.windows[0].close()

In [ ]:
# https://www.tutorialspoint.com/How-can-I-create-a-directory-if-it-does-not-exist-using-Python